https://www.kaggle.com/datasets/cdeotte/deberta-v3-small-finetuned-v1## ℹ️ Info

| Method | LB |
| --- | :---: |
| LGBM 5 fold | 0.801 |
| LGBM 15 fold | 0.802 |
| LGBM 15 fold + post-processing | 0.803 |
| LGBM 15 fold + 5 fold Deberta (Ensemble) | 0.807 |
| LGBM 16 fold + 5 fold Deberta (Ensemble) | 0.808 |
| LGBM 15 fold + 5 fold Deberta (Ensemble) + CountVectorizer | 0.810 |
| LGBM 15 fold + 5 fold Deberta (as features) + CountVectorizer | 0.811 |
| LGBM 16 fold + 5 fold Deberta (as features) + CountVectorizer + HashingVectorizer | 0.811 |
| LGBM 15 fold + 5 fold Deberta (as features) + TfidfVectorizer(ngram(3,6)) + CountVectorizer | 0.812 |
| LGBM 15 fold + (new)5 fold Deberta (as features) + TfidfVectorizer(ngram(3,6)) + CountVectorizer(ngram(3,5)) | 0.816 |
| LGBM 15 fold + (new)5 fold Deberta (as features) + TfidfVectorizer(ngram(3,6)) + CountVectorizer(ngram(3,4)) | 0.817 |
| LGBM 15 fold + (new)5 fold Deberta (as features) + more feature engineering + feature selection | 0.817 |


* 2024/04/15 : forked original great work kernels
    * https://www.kaggle.com/code/olyatsimboy/5-fold-deberta-lgbm
    * https://www.kaggle.com/code/aikhmelnytskyy/quick-start-lgbm
    * https://www.kaggle.com/code/hideyukizushi/aes2-5folddeberta-lgbm-countvectorizer-lb-810
    * https://www.kaggle.com/code/olyatsimboy/81-1-aes2-5folddeberta-lgbm-stacking  
    
    
* 2024/04/16 : ~~add HashingVectorizer~~ (not work)
* 2024/04/21 : Add MetaFEs. Train deberta-v3-large local (5Fold SKF) : https://www.kaggle.com/datasets/hideyukizushi/aes2-400-20240419134941    
* 2024/04/22 : change TfidfVectorizer ngram to (3,6), CountVectorizer ngram to (3,5)
* 2024/04/23 : change CountVectorizer ngram to (3,4)
* 2024/04/24 : MORE FEATURE ENGINEERING + FEATURE SELECTION : https://www.kaggle.com/code/xianhellg/more-feature-engineering-feature-selection-0-817
---

In [21]:
import pandas as pd
import gc
import pickle
import torch
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5,6,7,8" 
sys.path.append('/home/mcq/GitHub/aes2')
sys.path.append('/home/mcq/GitHub/aes2/topic-classificatio')
# import add_2_features
_test = pd.read_csv("/home/mcq/GitHub/aes2/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv")
# ENABLE_DONT_WASTE_YOUR_RUN_TIME = len(_test) < 10
ENABLE_DONT_WASTE_YOUR_RUN_TIME = False
if ENABLE_DONT_WASTE_YOUR_RUN_TIME:
    import shutil

#     shutil.copyfile("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv", "submission.csv")
#     exit(0)
    del _test
    gc.collect()

import torch
CUDA_AVAILABLE = torch.cuda.is_available()
print(f"{CUDA_AVAILABLE = }")

CUDA_AVAILABLE = True


In [22]:
# !cp /kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv submission.csv

# <div style="color:white;display:fill;border-radius:5px;background-color:seaGreen;text-align:center;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%">▶️ 5 Fold Deberta ◀️</div>

In [23]:
# fb_oof = pd.read_csv("/home/mcq/GitHub/aes2/submission_1.csv")
# fb_oof.shape

In [24]:
import xgboost as xgb
import pandas as pd 
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments, 
    DataCollatorWithPadding
)
from datasets import Dataset
from glob import glob
import gc
import torch
from scipy.special import softmax

MAX_LENGTH = 1024
TEST_DATA_PATH = "/home/mcq/GitHub/aes2/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv"
MODEL_PATH = '/home/mcq/GitHub/aes2/kaggle/input/aes2-400-20240419134941/*/*'
EVAL_BATCH_SIZE = 1

# Deberta Model

In [25]:
def get_deberta_predicted_score(df_test = None):
    models = glob(MODEL_PATH)
    tokenizer = AutoTokenizer.from_pretrained(models[0])

    def tokenize(sample):
        return tokenizer(sample['full_text'], max_length=MAX_LENGTH, truncation=True)
    if df_test is None:
        df_test = pd.read_csv(TEST_DATA_PATH)
    ds = Dataset.from_pandas(df_test).map(tokenize).remove_columns(['essay_id', 'full_text'])

    args = TrainingArguments(
        ".", 
        per_device_eval_batch_size=EVAL_BATCH_SIZE, 
        report_to="none"
    )

    predictions = []
    for model in models:
        model = AutoModelForSequenceClassification.from_pretrained(model)
        trainer = Trainer(
            model=model, 
            args=args, 
            data_collator=DataCollatorWithPadding(tokenizer), 
            tokenizer=tokenizer
        )

        preds = trainer.predict(ds).predictions
        predictions.append(softmax(preds, axis=-1))  
        del model, trainer
        torch.cuda.empty_cache()
        gc.collect()

    predicted_score = 0.

    for p in predictions:
        predicted_score += p

    predicted_score /= len(predictions)
    df_test['score'] = predicted_score.argmax(-1) + 1
    df_test.head()
    df_test[['essay_id', 'score']].to_csv('submission1.csv', index=False)
    return predicted_score

# <div style="color:white;display:fill;border-radius:5px;background-color:seaGreen;text-align:center;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%">▶️ Prize feedback ◀️</div>

In [26]:
def get_fb3_predicted(df_test = None):
    import pandas as pd
    import fb3_deberta_family_inference_9_28_updated

    if df_test is None:
        df_test = pd.read_csv(TEST_DATA_PATH)
    if len(df_test) < 10:
        fb3_predicted = fb3_deberta_family_inference_9_28_updated.predict_chunk(
            df_test.rename(columns={"essay_id": "text_id"})
        )
    else:
        fb3_predicted = fb3_deberta_family_inference_9_28_updated.predict(
            df_test.rename(columns={"essay_id": "text_id"})
        )
    return fb3_predicted

# <div style="color:white;display:fill;border-radius:5px;background-color:seaGreen;text-align:center;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%">▶️ 15 fold LGBM ◀️</div>

In [27]:
# Importing necessary libraries
import gc
import lightgbm as lgb
from sklearn.ensemble import VotingRegressor
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import spacy
import string
import random
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier,BaggingClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.metrics import cohen_kappa_score
from lightgbm import log_evaluation, early_stopping
from sklearn.linear_model import SGDClassifier
import polars as pl
import joblib

# Features engineering

In [28]:
def count_spelling_errors(text):
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_.lower() for token in doc]
    spelling_errors = sum(1 for token in lemmatized_tokens if token not in english_vocab)
    return spelling_errors

def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
def dataPreprocessing(x):
    # Convert words to lowercase
    x = x.lower()
    # Remove HTML
    x = removeHTML(x)
    # Delete strings starting with @
    x = re.sub("@\w+", '',x)
    # Delete Numbers
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    # Delete URL
    x = re.sub("http\w+", '',x)
    # Replace consecutive empty spaces with a single space character
    x = re.sub(r"\s+", " ", x)
    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
    # Remove empty characters at the beginning and end
    x = x.strip()
    return x

## Paragraph Features

In [29]:
# paragraph features
def remove_punctuation(text):
    """
    Remove all punctuation from the input text.
    
    Args:
    - text (str): The input text.
    
    Returns:
    - str: The text with punctuation removed.
    """
    # string.punctuation
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def Paragraph_Preprocess(tmp):
    # Expand the paragraph list into several lines of data
    tmp = tmp.explode('paragraph')
    # Paragraph preprocessing
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(dataPreprocessing))
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(remove_punctuation).alias('paragraph_no_pinctuation'))
    tmp = tmp.with_columns(pl.col('paragraph_no_pinctuation').map_elements(count_spelling_errors).alias("paragraph_error_num"))
    # Calculate the length of each paragraph
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x)).alias("paragraph_len"))
    # Calculate the number of sentences and words in each paragraph
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x.split('.'))).alias("paragraph_sentence_cnt"),
                    pl.col('paragraph').map_elements(lambda x: len(x.split(' '))).alias("paragraph_word_cnt"),)
    return tmp
# feature_eng

def Paragraph_Eng(train_tmp):
    num_list = [0, 50,75,100,125,150,175,200,250,300,350,400,500,600]
    num_list2 = [0, 50,75,100,125,150,175,200,250,300,350,400,500,600,700]
    aggs = [
        # Count the number of paragraph lengths greater than and less than the i-value
        *[pl.col('paragraph').filter(pl.col('paragraph_len') >= i).count().alias(f"paragraph_{i}_cnt") for i in [0, 50,75,100,125,150,175,200,250,300,350,400,500,600,700] ], 
        *[pl.col('paragraph').filter(pl.col('paragraph_len') <= i).count().alias(f"paragraph_{i}_cnt") for i in [25,49]], 
        # other
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in paragraph_fea2],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in paragraph_fea2],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in paragraph_fea2],
        *[pl.col(fea).sum().alias(f"{fea}_sum") for fea in paragraph_fea2],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in paragraph_fea2],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in paragraph_fea2],
        *[pl.col(fea).kurtosis().alias(f"{fea}_kurtosis") for fea in paragraph_fea2],
        *[pl.col(fea).quantile(0.25).alias(f"{fea}_q1") for fea in paragraph_fea2],  
        *[pl.col(fea).quantile(0.75).alias(f"{fea}_q3") for fea in paragraph_fea2],  
        ]
    
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

## Sentence Features

In [30]:
# sentence feature
def Sentence_Preprocess(tmp):
    # Preprocess full_text and use periods to segment sentences in the text
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=".").alias("sentence"))
    tmp = tmp.explode('sentence')
    # Calculate the length of a sentence
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x)).alias("sentence_len"))
    # Filter out the portion of data with a sentence length greater than 15
    tmp = tmp.filter(pl.col('sentence_len')>=15)
    # Count the number of words in each sentence
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x.split(' '))).alias("sentence_word_cnt"))
    
    return tmp
# feature_eng
sentence_fea = ['sentence_len','sentence_word_cnt']
def Sentence_Eng(train_tmp):
    aggs = [
        # Count the number of sentences with a length greater than i
        *[pl.col('sentence').filter(pl.col('sentence_len') >= i).count().alias(f"sentence_{i}_cnt") for i in [0,15,50,100,150,200,250,300] ], 
        *[pl.col('sentence').filter(pl.col('sentence_len') <= i).count().alias(f"sentence_<{i}_cnt") for i in [15,50] ], 
        # other
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in sentence_fea],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in sentence_fea],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in sentence_fea],
        *[pl.col(fea).sum().alias(f"{fea}_sum") for fea in sentence_fea],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in sentence_fea],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in sentence_fea],
        *[pl.col(fea).kurtosis().alias(f"{fea}_kurtosis") for fea in sentence_fea],
        *[pl.col(fea).quantile(0.25).alias(f"{fea}_q1") for fea in sentence_fea], 
        *[pl.col(fea).quantile(0.75).alias(f"{fea}_q3") for fea in sentence_fea], 
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

## Word Features

In [31]:
# word feature
def Word_Preprocess(tmp):
    # Preprocess full_text and use spaces to separate words from the text
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=" ").alias("word"))
    tmp = tmp.explode('word')
    # Calculate the length of each word
    tmp = tmp.with_columns(pl.col('word').map_elements(lambda x: len(x)).alias("word_len"))
    # Delete data with a word length of 0
    tmp = tmp.filter(pl.col('word_len')!=0)
    
    return tmp
# feature_eng
def Word_Eng(train_tmp):
    aggs = [
        # Count the number of words with a length greater than i+1
        *[pl.col('word').filter(pl.col('word_len') >= i+1).count().alias(f"word_{i+1}_cnt") for i in range(15) ], 
        # other
        pl.col('word_len').max().alias(f"word_len_max"),
        pl.col('word_len').mean().alias(f"word_len_mean"),
        pl.col('word_len').std().alias(f"word_len_std"),
        pl.col('word_len').quantile(0.25).alias(f"word_len_q1"),
        pl.col('word_len').quantile(0.50).alias(f"word_len_q2"),
        pl.col('word_len').quantile(0.75).alias(f"word_len_q3"),
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

# Set up loss functions

In [32]:
# idea from https://www.kaggle.com/code/rsakata/optimize-qwk-by-lgb/notebook#QWK-objective
def quadratic_weighted_kappa(y_true, y_pred):
    if isinstance(y_pred, xgb.QuantileDMatrix):
        # XGB
        y_true, y_pred = y_pred, y_true

        y_true = (y_true.get_label() + a).round()
        y_pred = (y_pred + a).clip(1, 6).round()
        qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")
        return 'QWK', qwk

    else:
        # For lgb
        y_true = y_true + a
        y_pred = (y_pred + a).clip(1, 6).round()
        qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")
        return 'QWK', qwk, True
def qwk_obj(y_true, y_pred):
    labels = y_true + a
    preds = y_pred + a
    preds = preds.clip(1, 6)
    f = 1/2*np.sum((preds-labels)**2)
    g = 1/2*np.sum((preds-a)**2+b)
    df = preds - labels
    dg = preds - a
    grad = (df/g - f*dg/g**2)*len(labels)
    hess = np.ones(len(labels))
    return grad, hess

a = 2.998
b = 1.092

# Feature Selection

In [38]:
def feature_select_wrapper():
    """
    lgm
    :param train
    :param test
    :return
    """
    # Part 1.
    print('feature_select_wrapper...')
    features = feature_names

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    fse = pd.Series(0, index=features)
         
    for train_index, test_index in skf.split(X, y_split):

        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold, y_test_fold_int = y[train_index], y[test_index], y_split[test_index]

        model = lgb.LGBMRegressor(
                    objective = qwk_obj,
                    metrics = 'None',
                    learning_rate = 0.05,
                    max_depth = 5,
                    num_leaves = 10,
                    colsample_bytree=0.3,
                    reg_alpha = 0.7,
                    reg_lambda = 0.1,
                    n_estimators=700,
                    random_state=412,
                    extra_trees=True,
                    class_weight='balanced',
                    verbosity = - 1)

        predictor = model.fit(X_train_fold,
                              y_train_fold,
                              eval_names=['train', 'valid'],
                              eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
                              eval_metric=quadratic_weighted_kappa,
                              callbacks=callbacks)
        models.append(predictor)
        predictions_fold = predictor.predict(X_test_fold)
        predictions_fold = predictions_fold + a
        predictions_fold = predictions_fold.clip(1, 6).round()
        predictions.append(predictions_fold)
        f1_fold = f1_score(y_test_fold_int, predictions_fold, average='weighted')
        f1_scores.append(f1_fold)

        kappa_fold = cohen_kappa_score(y_test_fold_int, predictions_fold, weights='quadratic')
        kappa_scores.append(kappa_fold)

#         cm = confusion_matrix(y_test_fold_int, predictions_fold, labels=[x for x in range(1,7)])

#         disp = ConfusionMatrixDisplay(confusion_matrix=cm,
#                                       display_labels=[x for x in range(1,7)])
#         disp.plot()
#         plt.show()
        print(f'F1 score across fold: {f1_fold}')
        print(f'Cohen kappa score across fold: {kappa_fold}')
        feature_importances = predictor.feature_importances_[:len(features)] # 不知道为什么predictor.feature_importances_会比features还长所以加这么一句强行使其一致
        fse += pd.Series(feature_importances, features)
#         if ENABLE_DONT_WASTE_YOUR_RUN_TIME:
#             break
    with open("fse.pickle", "wb") as f:
        pickle.dump(fse, f)
    
    # Part 4.
    feature_select = fse.sort_values(ascending=False).index.tolist()[:13000]
    print('done')
    return feature_select

In [34]:
# add2_feats = pd.read_pickle('/home/mcq/GitHub/aes2/train_data/add2-feat.pkl')
# add2_feats.head(5)
# for i in range(2):
#     train_feats[f'argument_{i}'] = train_feats_argument.iloc[:, i]
# c = train_feats.columns[~train_feats.columns.isin(feature_names)]
# train_feats = train_feats.drop['essay_id','score']

# 跑基础特征

In [35]:
columns = [  
    (
        pl.col("full_text").str.split(by="\n\n").alias("paragraph")
    ),
]
PATH = "kaggle/input/learning-agency-lab-automated-essay-scoring-2/"
paragraph_fea = ['paragraph_len','paragraph_sentence_cnt','paragraph_word_cnt']
paragraph_fea2 = ['paragraph_error_num'] + paragraph_fea

# # Load training and testing sets, while using \ n \ n character segmentation to list and renaming to paragraph for full_text data
# data1 = pd.read_csv(PATH + "train.csv")
# # #data1 = pd.read_csv(PATHS.train_path)
# data2 = pd.read_csv('/home/mcq/GitHub/aes2/train_data/persuade_2.0.csv')
# data2=data2[['essay_id_comp','full_text','holistic_essay_score']].rename(columns={'essay_id_comp':'essay_id','holistic_essay_score':'score'})

# train_pd=pd.concat([data1,data2],axis=0)
# train_pd=train_pd.drop_duplicates(subset=['full_text'],keep='first').reset_index(drop=True)

# train = pl.from_pandas(train_pd).with_columns(columns)
# Load training and testing sets using polars
# data1 = pl.read_csv(PATH + "train.csv")
# data2 = pl.read_csv('/home/mcq/GitHub/aes2/train_data/persuade_2.0.csv')

# # Select and rename columns in data2
# data2 = data2.select([
#     pl.col('essay_id_comp').alias('essay_id'),
#     pl.col('full_text'),
#     pl.col('holistic_essay_score').alias('score')
# ])

# # Concatenate the dataframes
# train = pl.concat([data1, data2])

# # Drop duplicates and reset index
# train = train.unique(subset=['full_text']).with_row_count('index').with_columns(columns)
train_df = pd.read_csv(PATH + "train.csv")
#data1 = pd.read_csv(PATHS.train_path)
persuade = pd.read_csv('/home/mcq/GitHub/aes2/train_data/persuade_2.0.csv')
persuade["label"] =  persuade["prompt_name"].astype("category").cat.codes
intersection = pd.merge(train_df, persuade, on="full_text", how="inner")[["essay_id", "full_text", "score", "prompt_name","label"]].reset_index(drop=True)
persuade.rename(columns={'essay_id_comp': 'essay_id'}, inplace=True)
persuade.rename(columns={'holistic_essay_score': 'score'}, inplace=True)

difference = train_df[~train_df["essay_id"].isin(intersection["essay_id"])].reset_index(drop=True)

difference1 = persuade[~persuade["essay_id"].isin(intersection["essay_id"])].reset_index(drop=True)
difference1 = difference1.iloc[:, :3] 
skf_train = pd.concat([train_df, difference1], axis=0, ignore_index=True) 
### skf_train就是加过额外数据集的train了， 



difference2 = persuade[~persuade["essay_id"].isin(intersection["essay_id"])].reset_index(drop=True)

mask = difference2['label'].isin([1,  5 , 0 , 7 , 6 , 8 ,14])
df_dropped = difference2[~mask]
GFX_add7 = difference2[~difference2["essay_id"].isin(df_dropped["essay_id"])].reset_index(drop=True)
##GFX_add7长度为12873，是在7个topic之内的点

GKF_train = df_dropped ###长度为13121,是topic之外的点
GKF_train = GKF_train.iloc[:, :3]  ##数据规范化处理

topic = pd.read_pickle("/home/mcq/GitHub/aes2/train_data/base_skf_feat/category-feat.pkl")
groups_1= topic['topic'] ## 读取一万七的topic

groups_2 = GFX_add7['label'] ##读取额外的topic
groups = pd.concat([groups_1, groups_2], axis=0, ignore_index=True)

GFX_add7 = GFX_add7.iloc[:, :3]

train_2 = pd.concat([train_df, GFX_add7], axis=0, ignore_index=True) ###把额外的7个topic之内的代码整合进原有的数据集，这里的代码结构和train一样
train_2 = train_2[train_2['essay_id'] != '1.51E+11'] # Pandas
train = pl.from_pandas(train_2).with_columns(columns) # Polars


# data2 = pd.read_csv('/home/mcq/GitHub/aes2/train_data/persuade_2.0.csv')
# data2=data2[['essay_id_comp','full_text','holistic_essay_score']].rename(columns={'essay_id_comp':'essay_id','holistic_essay_score':'score'})

# train=pd.concat([data1,data2],axis=0)
# train=train.drop_duplicates(subset=['full_text'],keep='first').reset_index(drop=True)




# train2 = pl.read_csv(PATH + "persuade_2.0.csv").with_columns(columns)
# train = combined_train = pl.concat([train1, train2])
test = pl.read_csv(PATH + "test.csv").with_columns(columns)






# TfidfVectorizer parameter
vectorizer = TfidfVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(3,6),
            min_df=0.05,
            max_df=0.95,
            sublinear_tf=True,
)
vectorizer.fit([i for i in train['full_text']])

vectorizer_cnt = CountVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(2,3),
            min_df=0.10,
            max_df=0.85,
)
vectorizer_cnt.fit([i for i in train['full_text']])

nlp = spacy.load("en_core_web_sm")
with open('/home/mcq/GitHub/aes2/kaggle/input/english-word-hx/words.txt', 'r') as file:
    english_vocab = set(word.strip().lower() for word in file)

# Display the first sample data in the training set
train.head(1)


# if __name__ == "__main__":
    

essay_id,full_text,score,paragraph
str,str,i64,list[str]
"""000d118""","""Many people have car where the…",3,"[""Many people have car where they live. The thing they don't know is that when you use a car alot of thing can happen like you can get in accidet or the smoke that the car has is bad to breath on if someone is walk but in VAUBAN,Germany they dont have that proble because 70 percent of vauban's families do not own cars,and 57 percent sold a car to move there. Street parkig ,driveways and home garages are forbidden on the outskirts of freiburd that near the French and Swiss borders. You probaly won't see a car in Vauban's streets because they are completely ""car free"" but If some that lives in VAUBAN that owns a car ownership is allowed,but there are only two places that you can park a large garages at the edge of the development,where a car owner buys a space but it not cheap to buy one they sell the space for you car for $40,000 along with a home. The vauban people completed this in 2006 ,they said that this an example of a growing trend in Europe,The untile states and some where else are suburban life from auto use this is called ""smart planning"". The current efforts to drastically reduce greenhouse gas emissions from tailes the passengee cars are responsible for 12 percent of greenhouse gas emissions in Europe and up to 50 percent in some car intensive in the United States. I honeslty think that good idea that they did that is Vaudan because that makes cities denser and better for walking and in VAUBAN there are 5,500 residents within a rectangular square mile. In the artical David Gold berg said that ""All of our development since World war 2 has been centered on the cars,and that will have to change"" and i think that was very true what David Gold said because alot thing we need cars to do we can go anyway were with out cars beacuse some people are a very lazy to walk to place thats why they alot of people use car and i think that it was a good idea that that they did that in VAUBAN so people can see how we really don't need car to go to place from place because we can walk from were we need to go or we can ride bycles with out the use of a car. It good that they are doing that if you thik about your help the earth in way and thats a very good thing to. In the United states ,the Environmental protection Agency is promoting what is called ""car reduced""communtunties,and the legislators are starting to act,if cautiously. Maany experts expect pubic transport serving suburbs to play a much larger role in a new six years federal transportation bill to approved this year. In previous bill,80 percent of appropriations have by law gone to highways and only 20 percent to other transports. There many good reason why they should do this. ""]"


In [16]:
print(train_2.shape)
GKF_train = GKF_train[GKF_train['essay_id'] != '1.51E+11']
print(GKF_train.shape)

with open("/home/mcq/GitHub/aes2/train_data/train_2.pkl", "wb") as f:
    pickle.dump(train_2, f)

with open("/home/mcq/GitHub/aes2/train_data/GKF_train.pkl", "wb") as f:
    pickle.dump(GKF_train, f)

(30178, 3)
(13121, 3)


In [17]:
# train_feats_id = train_feats.iloc[:,0]
# train_id = train.to_pandas().iloc[:,0]
# set_train_feats_id = set(train_feats_id)
# set_train_id = set(train_id)

# # 找到不同的值
# diff_1 = set_train_feats_id.difference(set_train_id)
# diff_2 = set_train_id.difference(set_train_feats_id)

# # 合并结果
# diff = diff_1.union(diff_2)

# # 打印不同的值
# print("不同的值:\n", diff)

# difference = set(train_id).difference(set(train_feats_id))
# print(difference)


# # 找出重复值
# duplicates = train_id[train_id.duplicated()]

# # 只显示唯一的重复值
# unique_duplicates = duplicates.unique()

# print("唯一的重复值：")
# print(unique_duplicates)


In [18]:
tmp = Paragraph_Preprocess(train)
train_feats = Paragraph_Eng(tmp)

# train_feats['score'] = train['score']
# # Obtain feature names
# feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
# print('Features Number: ',len(feature_names))
# train_feats.head(3)

tmp = Sentence_Preprocess(train)
train_feats = train_feats.merge(Sentence_Eng(tmp), on='essay_id', how='left')

tmp = Word_Preprocess(train)
train_feats = train_feats.merge(Word_Eng(tmp), on='essay_id', how='left')


# feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
# print('Features Number: ',len(feature_names))
# train_feats.head(3)


    # TF-IDF
train_tfid = vectorizer.transform([i for i in train['full_text']])
# Convert to array
dense_matrix = train_tfid.toarray()
# Convert to dataframe
df = pd.DataFrame(dense_matrix)
# rename features
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = train_feats['essay_id']
# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(df, on='essay_id', how='left')

# feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
# print('Features Number: ',len(feature_names))
# train_feats.head(3)

# Count
train_tfid = vectorizer_cnt.transform([i for i in train['full_text']])
dense_matrix = train_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_cnt_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = train_feats['essay_id']
train_feats = train_feats.merge(df, on='essay_id', how='left')

# feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
# print('Features Number: ',len(feature_names))
# train_feats.head(3)

# add Deberta predictions to LGBM as features
# deberta_oof = joblib.load('/home/mcq/GitHub/aes2/kaggle/input/aes2-400-20240419134941/oof.pkl')
# print(deberta_oof.shape, train_feats.shape)

# for i in range(6):
#     train_feats[f'deberta_oof_{i}'] = deberta_oof[:, i]

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ', len(feature_names))    

print(f"{train_feats.shape=}")



/tmp/ipykernel_173943/1952907439.py:20: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  tmp = tmp.with_columns(pl.col('paragraph').map_elements(dataPreprocessing))
/tmp/ipykernel_173943/1952907439.py:21: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  tmp = tmp.with_columns(pl.col('paragraph').map_elements(remove_punctuation).alias('paragraph_no_pinctuation'))
/tmp/ipykernel_173943/1952907439.py:22: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  tmp = tmp.with_columns(pl.col('paragraph_no_pinctuation').map_elements(count_spelling_errors).alias("paragraph_error_num"))
/tmp/ipykernel_173943/1952907439.py:24: MapWithoutRet

Features Number:  53


sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
/tmp/ipykernel_173943/1351511948.py:7: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x)).alias("sentence_len"))
/tmp/ipykernel_173943/1351511948.py:11: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x.split(' '))).alias("sentence_word_cnt"))
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
/tmp/ipykernel_

Features Number:  102
Features Number:  19773
Features Number:  21952
Features Number:  21952
train_feats.shape=(30178, 21954)


# 载入基础特征

In [19]:
# 存入基础特征
with open("/home/mcq/GitHub/aes2/train_data/basic_feats.pkl", "wb") as f:
    pickle.dump(train_feats, f)
with open("/home/mcq/GitHub/aes2/train_data/basic_feature_names.pkl", "wb") as f:
    pickle.dump(feature_names, f)

In [ ]:
# 载入基础特征
feature_names = list()
with open("/home/mcq/GitHub/aes2/train_data/basic_feats.pkl", "rb") as f:
    train_feats = pickle.load(f)
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ', len(feature_names)) 
print(f"{train_feats.shape=}")

Features Number:  21952
train_feats.shape=(30178, 21954)


In [ ]:
# fb_oof_0 = pd.read_csv("/home/mcq/GitHub/aes2/train_data/fb3_feat_0.csv")
# fb_oof_0.columns = fb_oof_0.columns + '_0'
# train_feats = pd.merge(train_feats, fb_oof_0, left_on="essay_id", right_on="text_id_0").drop("text_id_0", axis=1)
# feature_names += list(fb_oof_0.columns.drop("text_id_0"))
# fb_oof_1 = pd.read_csv("/home/mcq/GitHub/aes2/train_data/fb3_feat_1.csv")
# fb_oof_1.columns = fb_oof_1.columns + '_1'
# train_feats = pd.merge(train_feats, fb_oof_1, left_on="essay_id", right_on="text_id_1").drop("text_id_1", axis=1)
# feature_names += list(fb_oof_1.columns.drop("text_id_1"))

# fb_oof_2 = pd.read_csv("/home/mcq/GitHub/aes2/train_data/fb3_feat_2.csv")
# fb_oof_2.columns = fb_oof_2.columns + '_2'
# train_feats = pd.merge(train_feats, fb_oof_2, left_on="essay_id", right_on="text_id_2").drop("text_id_2", axis=1)
# feature_names += list(fb_oof_2.columns.drop("text_id_2"))
# print('Features Number: ', len(feature_names)) 
# print(f"{train_feats.shape=}")

In [ ]:
# train_feats_argument = pd.read_pickle('/home/mcq/GitHub/aes2/train_data/argument-feat.pkl')
# for i in range(2):
#     train_feats[f'argument_{i}'] = train_feats_argument.iloc[:, i]
#     feature_names += list([f'argument_{i}'])



# # add2_feats = pd.read_pickle('/home/mcq/GitHub/aes2/train_data/add2-feat.pkl')
# # # add2_feats = add2_feats.drop(columns=['paragraph_processed'])
# # train_feats = pd.concat([train_feats, add2_feats], axis = 1)
# # feature_names += list(add2_feats.columns)
# print('Train Feats:', train_feats.shape)
# print('All Feature Names: ',len(feature_names))

In [ ]:
# category_feat = pd.read_pickle('/home/mcq/GitHub/aes2/train_data/category-feat.pkl')
# train_feats = pd.merge(train_feats, category_feat, on="essay_id",how='inner')
# feature_names += list(category_feat.columns.drop('essay_id'))
# print('Train Feats:', train_feats.shape)
# print('All Feature Names: ',len(feature_names))

In [ ]:
import argument_classifier
import topicfeaturesmall

predicted_score = get_deberta_predicted_score(train_2)
with open("/home/mcq/GitHub/aes2/train_data/deberta_feats.pkl", "wb") as f:
    pickle.dump(predicted_score, f)
for i in range(6):
    train_feats[f'deberta_oof_{i}'] = predicted_score[:, i]

tokenizers.__version__: 0.19.1
transformers.__version__: 4.41.1


Map:   0%|          | 0/30178 [00:00<?, ? examples/s]

In [ ]:
fb3_predicted = get_fb3_predicted(train_2)
# fb3_predicted.shape
with open("/home/mcq/GitHub/aes2/train_data/fb3_feats.pkl", "wb") as f:
    pickle.dump(fb3_predicted, f)

tokenizers.__version__: 0.19.1
transformers.__version__: 4.41.1


Model: microsoft/deberta-v3-base Score: 0.4590  Scores: [0.4927071121286561, 0.44975919653806296, 0.41900989606208666, 0.46100316932040336, 0.47875430698135113, 0.4529461925915486]


0    000d118
1    000fe60
2    001ab80
3    001bdc0
4    002ba53
Name: text_id, dtype: object
0    000d118
1    000fe60
2    001ab80
3    001bdc0
4    002ba53
Name: text_id, dtype: object
CustomModel(
  (model): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV

  0%|          | 0/126 [00:01<?, ?it/s]

0    2ce625f
1    2ce7f8d
2    2ce9900
3    2cf189d
4    2cf20ff
Name: text_id, dtype: object
0    2ce625f
1    2ce7f8d
2    2ce9900
3    2cf189d
4    2cf20ff
Name: text_id, dtype: object
CustomModel(
  (model): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV

...

  0%|          | 0/126 [00:01<?, ?it/s]

0    1111156C5EB8
1    5C2B6A9275A3
2    7AA2C0834BE1
3    CFC1F5729C6D
4    3DA49C7B4B14
Name: text_id, dtype: object
0    1111156C5EB8
1    5C2B6A9275A3
2    7AA2C0834BE1
3    CFC1F5729C6D
4    3DA49C7B4B14
Name: text_id, dtype: object
CustomModel(
  (model): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDrop

  0%|          | 0/126 [00:01<?, ?it/s]

0    7D36939006E9
1    48F2F2876E3D
2    6D1EF2E394FB
3    2945D26BEE91
4    BDA453D8E1C7
Name: text_id, dtype: object
0    7D36939006E9
1    48F2F2876E3D
2    6D1EF2E394FB
3    2945D26BEE91
4    BDA453D8E1C7
Name: text_id, dtype: object
CustomModel(
  (model): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDrop

  0%|          | 0/126 [00:01<?, ?it/s]

In [ ]:
argument_predicted = argument_classifier.predict_chunk(train_2)

with open("/home/mcq/GitHub/aes2/train_data/aargument_feats.pkl", "wb") as f:
    pickle.dump(argument_predicted, f)

for i in range(2):
    train_feats[f'argument_{i}'] = argument_predicted.iloc[:, i]

Map:   0%|          | 0/30178 [00:00<?, ? examples/s]

In [ ]:
train_feats = pd.merge(
    train_feats,
    fb3_predicted,
    left_on="essay_id",
    right_on="text_id"
).drop("text_id", axis=1)

# Features number
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))

# 读取所有train feats 

In [40]:
with open("/home/mcq/GitHub/aes2/train_data/train_feats.pkl", "rb") as f:
    train_feats = pickle.load(f)
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ', len(feature_names)) 
print(f"{train_feats.shape=}")
with open("/home/mcq/GitHub/aes2/train_data/y.pkl", "rb") as f:
    y = pickle.load(f)
with open("/home/mcq/GitHub/aes2/train_data/y_split.pkl", "rb") as f:
    y_split = pickle.load(f)

Features Number:  21913
train_feats.shape=(30178, 21914)


In [ ]:
# ttc = train_feats.select_dtypes(include=['object']).columns
# train_feats= train_feats.drop(columns=['essay_id', 'paragraph_processed'])
# rmv = ['essay_id', 'paragraph_processed']
# feature_names = [col for col in feature_names if col not in rmv]
# train_feats = train_feats.drop('essay_id')
# feature_names = feature_names.drop(['essay_id','paragraph_processed'])
# print('Train Feats:', train_feats.shape)
# print('All Feature Names: ',len(feature_names))

In [ ]:
# Converting the 'text' column to string type and assigning to X
X = train_feats[feature_names].astype(np.float32).values

# Converting the 'score' column to integer type and assigning to y
y_split = train_feats['score'].astype(int).values
y = train_feats['score'].astype(np.float32).values-a



In [41]:
f1_scores = []
kappa_scores = []
models = []
predictions = []
callbacks = [log_evaluation(period=25), early_stopping(stopping_rounds=75,first_metric_only=True)]

feature_select = feature_select_wrapper()

feature_select_wrapper...
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=5) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=32) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.808265	valid's QWK: 0.81157
[50]	train's QWK: 0.873475	valid's QWK: 0.87614
[75]	train's QWK: 0.88674	valid's QWK: 0.890012
[100]	train's QWK: 0.890722	valid's QWK: 0.892374
[125]	train's QWK: 0.893974	valid's QWK: 0.896185
[150]	train's QWK: 0.897196	valid's QWK: 0.898985
[175]	train's QWK: 0.900718	valid's QWK: 0.901184
[200]	train's QWK: 0.902589	valid's QWK: 0.902617
[225]	train's QWK: 0.904201	valid's QWK: 0.903739
[250]	train's QWK: 0.906476	valid's QWK: 0.904127
[275]	train's QWK: 0.90816	valid's QWK: 0.905263
[300]	train's

In [42]:
X = train_feats[feature_select].astype(np.float32).values

print('Features Select Number: ', len(feature_select))

with open("/home/mcq/GitHub/aes2/train_data/train_feats.pkl", "wb") as f:
    pickle.dump(train_feats, f)
with open("/home/mcq/GitHub/aes2/train_data/feature_select.pkl", "wb") as f:
    pickle.dump(feature_select, f)
with open("/home/mcq/GitHub/aes2/train_data/X.pkl", "wb") as f:
    pickle.dump(X, f)
with open("/home/mcq/GitHub/aes2/train_data/y.pkl", "wb") as f:
    pickle.dump(y, f)
with open("/home/mcq/GitHub/aes2/train_data/y_split.pkl", "wb") as f:
    pickle.dump(y_split, f)

Features Select Number:  13000


# Test Pipeline

In [ ]:
# if ENABLE_DONT_WASTE_YOUR_RUN_TIME:
#     import shutil

#     shutil.copyfile("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv", "submission.csv")
def preprocess_test(test: pl.DataFrame| None = None) -> pd.DataFrame:
    # import sys
    # import os
    # sys.path.append('/kaggle/input/argument-classifier')
    import argument_classifier
    
    if test is None:
        test = pl.read_csv(PATH + "test.csv").with_columns(columns)
    tmp = Paragraph_Preprocess(test)
    test_feats = Paragraph_Eng(tmp)
    # Sentence
    tmp = Sentence_Preprocess(test)
    test_feats = test_feats.merge(Sentence_Eng(tmp), on='essay_id', how='left')
    # Word
    tmp = Word_Preprocess(test)
    test_feats = test_feats.merge(Word_Eng(tmp), on='essay_id', how='left')

    # TfidfVectorizer
    test_tfid = vectorizer.transform([i for i in test['full_text']])
    dense_matrix = test_tfid.toarray()
    df = pd.DataFrame(dense_matrix)
    tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
    df.columns = tfid_columns
    df['essay_id'] = test_feats['essay_id']
    test_feats = test_feats.merge(df, on='essay_id', how='left')

    # CountVectorizer
    test_tfid = vectorizer_cnt.transform([i for i in test['full_text']])
    dense_matrix = test_tfid.toarray()
    df = pd.DataFrame(dense_matrix)
    tfid_columns = [ f'tfid_cnt_{i}' for i in range(len(df.columns))]
    df.columns = tfid_columns
    df['essay_id'] = test_feats['essay_id']
    test_feats = test_feats.merge(df, on='essay_id', how='left')

    # HashingVectorizer
    # test_tfid = vectorizer_hash.transform([i for i in test['full_text']])
    # dense_matrix = test_tfid.toarray()
    # df = pd.DataFrame(dense_matrix)
    # tfid_columns = [ f'tfid_cnt_{i}' for i in range(len(df.columns))]
    # df.columns = tfid_columns
    # df['essay_id'] = test_feats['essay_id']
    # test_feats = test_feats.merge(df, on='essay_id', how='left')
    
    
    
    
    predicted_score = get_deberta_predicted_score()
    
    for i in range(6):
        test_feats[f'deberta_oof_{i}'] = predicted_score[:, i]
    fb3_predicted = get_fb3_predicted()
    fb3_predicted.shape

    argument_predicted = argument_classifier.predict_chunk(train = pd.read_csv(TEST_DATA_PATH))
    for i in range(2):
        test_feats[f'argument_{i}'] = argument_predicted.iloc[:, i]
        
    
    test_feats = pd.merge(
        test_feats,
        fb3_predicted,
        left_on="essay_id",
        right_on="text_id"
    ).drop("text_id", axis=1)
    
    # Features number
    feature_names = list(filter(lambda x: x not in ['essay_id','score'], test_feats.columns))
    print('Features number: ',len(feature_names))
    test_feats.head(3)
    return test_feats[feature_select]

def infer(test_feats, models):
    probabilities = []
    for model in models:
        proba = model.predict(test_feats) + a
        probabilities.append(proba)

    # Compute the average probabilities across all models
    predictions = np.mean(probabilities, axis=0)
    predictions = np.round(predictions.clip(1, 6))

    # Print the predictions
    print(predictions)

    submission = pd.read_csv("/home/mcq/GitHub/aes2/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv")
    submission['score'] = predictions
    submission['score'] = submission['score'].astype(int)
    submission.to_csv("submission.csv", index=None)
    display(submission.head())

if __name__ == "__main__":
    test_feats = preprocess_test()
    print(test_feats.head(5))

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

0    000d118
1    000fe60
2    001ab80
Name: text_id, dtype: object
0    000d118
1    000fe60
2    001ab80
Name: text_id, dtype: object
CustomModel(
  (model): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_featur

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread Exception ignored in: <function _ConnectionBase.__del__ at 0x7fd794122a70>
Traceback (most recent call last):
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/multiprocessing/connection.py", line 132, in __del__
QueueFeederThread:
Traceback (most recent call last):
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    self._close()
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/threading.py"

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Features number:  21966
   deberta_oof_0  deberta_oof_5  deberta_oof_2  deberta_oof_4  deberta_oof_1  \
0       0.021534       0.000785       0.529225       0.002071       0.379466   
1       0.001421       0.000288       0.839511       0.000765       0.071248   
2       0.001564       0.013603       0.033057       0.433214       0.001929   

   deberta_oof_3  tfid_3108  tfid_cnt_1807  tfid_cnt_263  tfid_cnt_1805  ...  \
0       0.066919   0.000000              0             0              3  ...   
1       0.086768   0.012754              1             1              0  ...   
2       0.516632   0.009019              0             4              2  ...   

   tfid_8803  tfid_8804  tfid_8805  tfid_8806  tfid_8807  tfid_8809  \
0        0.0   0.019419   0.000000   0.013999        0.0   0.013413   
1        0.0   0.019464   0.000000   0.000000        0.0   0.016664   
2        0.0   0.013765   0.015706   0.012300        0.0   0.000000   

   tfid_8811  tfid_8812  tfid_12853  tfid_12857  

# <div style="color:white;display:fill;border-radius:5px;background-color:seaGreen;text-align:center;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%">▶️ Deberta Ensemble ◀️</div>

In [ ]:
# !cat submission.csv
# df1 = pd.read_csv('/kaggle/working/submission_1.csv')
# df2 = pd.read_csv('/kaggle/working/submission_2.csv')

# # Merging the dataframes on 'essay_id'
# df = pd.merge(left=df1, right=df2, on='essay_id', suffixes=('_1', '_2'))

# # Calculating the average score directly without apply()
# df['score'] = ((df['score_1'] + df['score_2']) / 2).round().astype(int)

# # Saving the desired columns to a new csv file
# df[['essay_id', 'score']].to_csv('submission.csv', index=False)